In [1]:
# Using Language Models
from langchain.llms.openai import OpenAI #llm 모델 선택 (davinci 는 비싸)
from langchain.chat_models import ChatOpenAI #chat 모델 선택, #ChatAnthropic

llm = OpenAI()
#chat = ChatOpenAI() # ChatAnthropic() 모델 바로 바꿀수 있어
# chat모델은 chat에 특화
a = llm.predict("How many planets are there")
#b = chat.predict("How many planets are there") 
a
#b

/Users/chulwonchae/FullStack_GPT/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/chulwonchae/FullStack_GPT/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


' in the solar system\nThere are eight planets in the solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. In addition, there are five dwarf planets: Ceres, Pluto, Haumea, Makemake, and Eris.'

In [2]:
from langchain.llms.openai import OpenAI #llm 모델 선택 (davinci 는 비싸)
from langchain.chat_models import ChatOpenAI #chat 모델 선택, #ChatAnthropic

#llm = OpenAI()
chat = ChatOpenAI() # ChatAnthropic() 모델 바로 바꿀수 있어
# chat모델은 chat에 특화
# a = llm.predict("How many planets are there")
b = chat.predict("How many planets are there") 
b

/Users/chulwonchae/FullStack_GPT/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/chulwonchae/FullStack_GPT/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

#### Prompt template
- Prompt templates help to translate user input and parameters into instriction for a language model. 
- Can be used to guid a model's response, helping it understand the context and generate relevant and coherent language based output.
- The output of prompt templates PromptValue, and this will be passed to an LLM or a ChatModel 

In [6]:
from langchain.chat_models import ChatOpenAI #chat 모델 선택, #ChatAnthropic
from langchain.prompts import PromptTemplate, ChatPromptTemplate #ChatPromptTemplate : 은 아래 'messages'로 부터 프롬프트를 만들고 / PromptTemplate : 은 string으로부터 만듬

chat = ChatOpenAI(temperature=0.1) 

template = PromptTemplate.from_template("What is the distance between {place_a} and {place_b}? Also, what is your name?")
prompt = template.format(place_a='Mexico', place_b='Thailand')

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles). My name is Assistant.'

#### ChatPromptTemplates
- These prompt templates are used to format a list of messages. These "templates" consist of a list of templates themselves. 

In [7]:
# # predict message
# from langchain.schema import HumanMessage, AIMessage, SystemMessage #system message : 우리가 llm설정들을 제공하기 위한 message
# messages = [
#     SystemMessage(content = "You are a geography expert. And you only reply in {language}"),
#     AIMessage(content = "안녕하세요, 저는 대한민국 대동여지도를 만든 {name} 입니다."),
#     HumanMessage(content = "What is the distance between {place_a} and {place_b}? Also, what is your name?")
# ]

# chat.predict_messages(messages)

- "system": 시스템 메시지를 나타내며, 주로 초기 설정이나 규칙을 지정하는 데 사용됩니다.
- "ai": 인공지능 모델이 생성한 응답을 나타냅니다.
- "human": 사용자(또는 인간)가 입력한 메시지를 나타냅니다

In [8]:
# tempplate 생성
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}"),
    ("ai", "안녕하세요, 저는 대한민국 대동여지도를 만든 {name} 입니다."),
    ("human", "What is the distance between {place_a} and {place_b}? Also, what is your name?")
])

prompt = template.format_messages(
    language = "Greek",
    name = "Socrates",
    place_a = "Mexico",
    place_b = "Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [9]:
# Output Parser필요한 이유는 llm의 응답을 변형해야할 때가 있기 때문 (리스트로 변환하던지)
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse("Hello,how, are,you")

['Hello', 'how', 'are', 'you']

In [10]:
template = ChatPromptTemplate.from_messages(
    [("system","You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",),
     ("human","{question}"),]
)
prompt = template.format_messages(max_items=10, question="What are the colors?")
result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['Red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [11]:
# Better way.
# 코드를 다시 정리 해보자 (여기서 부터 코드 다시 돌려봐)
chat = ChatOpenAI(temperature=0.1) 

class CommaOutputParser(BaseOutputParser):
    
    def parse(self,text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

template = ChatPromptTemplate.from_messages(
    [("system","You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else.",),
     ("human","{question}"),]
)

# Chain을 만들거야
chain = template | chat | CommaOutputParser()
chain.invoke({  #chain 실행, dictionary형태로
    "max_items" : 5,
    "question" : "What are the Pokemons"}) 

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

In [12]:
# 2 chain = chain +chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any types of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [13]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modifiy the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain # "recipe" context object를 체인에 넣어줘 (아래 코드 안쓰고, recipe 값에 결과 저장 후, 다음 체인으로 전달)
final_chain.invoke({ 
    "cuisine":"indian"
})

#"recipe": chef_chain 이것을 함으로서 자동으로 veg_chain.invoke() 가 돌아가는 것임

# # 즉, chef_chain의 결과 {recipe}를 veg_chain의 input 으로..
# veg_chain.invoke({
#     "recipe":"chatmodel" #이런식으로
# })

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular Indian dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons ground coriander
- 1 teaspoon turmeric
- 1 teaspoon garam masala
- 1 teaspoon paprika
- 1 teaspoon chili powder (adjust to taste)
- 3 cloves garlic, minced
- 1 tablespoon grated ginger
- Salt and pepper to taste
- 2 tablespoons oil
- 1 onion, diced
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, combine yogurt, lemon juice, cumin, coriander, turmeric, garam masala, paprika, chili powder, garlic, ginger, salt, and pepper. Add the chicken pieces and mix well to coat. Marinate for at least 1 hour, or overnight for best results.

2. Heat oil in a large

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can replace the chicken with a suitable alternative. One great option is to use paneer, which is a type of Indian cottage cheese that holds its shape well when cooked. Here's how you can prepare the paneer for this recipe:\n\nIngredients:\n- 1 lb paneer, cut into bite-sized cubes\n\nInstructions:\n1. In a bowl, combine yogurt, lemon juice, cumin, coriander, turmeric, garam masala, paprika, chili powder, garlic, ginger, salt, and pepper as per the original recipe.\n2. Add the paneer cubes to the marinade and mix gently to coat. Allow the paneer to marinate for at least 1 hour, or overnight for best results.\n3. Follow the remaining steps of the recipe as directed, substituting the marinated paneer for the chicken.\n4. When cooking the paneer in the skillet, be gentle to avoid breaking the cubes. Cook until the paneer is lightly browned on the edges.\n\nBy using paneer as a replacement for chicken, you can e

In [14]:
# # 모듈
# LangChain은 다음과 같은 주요 모듈을 위한 확장 가능한 표준 인터페이스 및 외부 통합 기능을 제공합니다:

# - Model I/O
# 언어 모델과의 인터페이스

# - Retrieval
# 애플리케이션별 데이터를 사용한 인터페이스

# - Agents
# 높은 수준의 지침이 주어지면 체인이 어떤 도구를 사용할지 선택하도록 합니다

# - Chains
# 일반적인 빌딩 블록 구성

# - Memory
# 체인 실행 간 지속적인 응용 프로그램 상태

# - Callbacks
# 모든 체인의 중간 단계 기록 및 스트리밍

# few shot 
# you are giving examples to model how to answer

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

t = PromptTemplate.from_template("What is the capital of {country}")
t.format(country='France')

'What is the capital of France'

In [16]:
# #이 형식도 똑같아 (langchain document way)
# from langchain.chat_models import ChatOpenAI
# from langchain.prompts import ChatMessagePromptTemplate
# from langchain.prompts.few_shot import FewShotPromptTemplate
# from langchain.callbacks import StreamingStdOutCallbackHandler

# chat = ChatOpenAI(
#     temperature=0.1, # 낮을수록 창의적
#     streaming=True,  # 실시간으로 글자 보여줘 
#     callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

# t = PromptTemplate(
#     template = "What is the capital of {country}",
#     input_variables = ['country'])

# t.format(country='France')

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate 
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

#example
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]
#format examples
# example_template = """Human: {question} AI : {answer}"""
# example_prompt = PromptTemplate.from_template(example_template)
# or just 
example_prompt = PromptTemplate.from_template("Human: {question}\n AI : {answer}")


prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Human: What do you know about {country}?", #example 다 넣고, question of user
    input_variables = ["country"]
)

#prompt.format(country="Germany")

chain = prompt | chat
chain.invoke({"country":"Germany"})

AI: 
        Here is what I know:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

In [19]:
# chatprompt templete 사용해보기
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

#example
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece", # 참고
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt =  ChatPromptTemplate.from_messages(
    [
    ("human", "What do you know about {country}?"), #위 참고 가 country
    ("ai", "{answer}")
])


example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)


final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert"),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat
chain.invoke({"country":"Germany"})


        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro\n        ')

In [20]:
#  Dynamic example selector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatMessagePromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector # 예제의 양을 얼마나 되는지 확인하고, 세팅값에 따라 알맞은 prompt예제 골라줄 것임.

        

chat = ChatOpenAI(
    temperature=0.1, # 낮을수록 창의적
    streaming=True,  # 실시간으로 글자 보여줘 
    callbacks=[StreamingStdOutCallbackHandler()]) # console에 보이는 문자를 다 print해줘 (다양한 llm에서 일어나는 event를 감지하는 방법)

#example
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\n AI : {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=80,
)

prompt = FewShotPromptTemplate( 
    example_prompt = example_prompt,
    example_selector=example_selector, #추가 (이제는 전체 예제를 사용하는게 아니라 selector를 거친 예제들만)
    suffix = "Human: What do you know about {country}?", 
    input_variables = ["country"]
)

prompt.format(country='Brazil')

'Human: What do you know about France?\n AI : \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [22]:
#  Dynamic example selector
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

# Serialization and Composition

In [24]:
# How to load the prompt templates from the disc. 
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")

'What is the capital of Germany'

In [26]:
# How to load the prompt templates from the disc. 
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

prompt.format(country="Germany")

'What is the capital of Germany'

In [27]:
# How to compose many prompt together 
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

#이거 끄고 아래꺼 돌려봐
full_prompt.format(
    character= "Pirate",
    example_question= "What is your location?",
    example_answer= "Arrrrg! That is a secret!! Arg arg!!",
    question= "What is your fav food?",
)


# chain = full_prompt | chat

# chain.invoke(
#     {
#         "character": "Pirate",
#         "example_question": "What is your location?",
#         "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
#         "question": "What is your fav food?",
#     }
# )

'\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n\n                                     \n    \n    This is an example of how you talk:\n\n    Human: What is your location?\n    You: Arrrrg! That is a secret!! Arg arg!!\n\n                              \n    \n    Start now!\n\n    Human: What is your fav food?\n    You:\n\n'

In [30]:
# Caching allows save the responses from llm. Save the questions and answers and re-use it when it is asked again. Save money. 
# Cache we are using is memory cache which means once we restart the notebook, it will be gone. 
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug 
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache()) # responses from llm, will be in cache memory
set_debug(True) #shows more detail what is going on

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\

In [29]:
# only took 0.2s
chat.predict("How do you make italian pasta")

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- Pinch of salt\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add a pinch of salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into desired shapes, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes, or until al dente.\n9. Drain the pasta and toss with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!'

In [31]:
# SQLiteCache
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug 
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("langchain.db")) # name anything 


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.74s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. 

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy

In [32]:
chat.predict("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut t

'To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide to making Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it is smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, roll out the dough using a pasta machine or a rolling pin until it is thin.\n7. Cut the dough into your desired shape, such as fettuccine or spaghetti.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al dente.\n9. Drain the pasta and toss it with your favorite sauce or toppings.\n\nEnjoy

In [34]:
#How much money are we spending on each call
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)
set_debug(False) 
with get_openai_callback() as usage: # usage 라고 안써도괜찮아
    chat.predict("What is the recope for soju")
    print(usage)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [36]:
#How much money are we spending on each call
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)
set_debug(False) 
with get_openai_callback() as usage: # usage 라고 안써도괜찮아
    a = chat.predict("What is the recope for soju")
    b = chat.predict("What is the recope for bread")
    print(a,b,"\n")
    print(usage)
    
#아래 request가 2개 이지만, 지금 langchain.db가 돌아가고 있어서 1개로 만 나옴

Ingredients:
- 1 bottle of soju
- Ice cubes
- Water or soda water (optional)
- Fresh fruit or herbs for garnish (optional)

Instructions:
1. Chill the bottle of soju in the refrigerator for at least 2 hours before serving.
2. Fill a glass with ice cubes.
3. Pour the desired amount of soju over the ice cubes.
4. If desired, top off with water or soda water to dilute the alcohol content.
5. Garnish with fresh fruit or herbs for added flavor and presentation.
6. Stir well before serving.
7. Enjoy your refreshing soju cocktail! Ingredients:
- 3 1/2 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups warm water
- 2 tablespoons sugar
- 1 teaspoon salt
- 2 tablespoons olive oil

Instructions:
1. In a small bowl, dissolve the yeast in warm water and let it sit for about 5 minutes until it becomes frothy.
2. In a large mixing bowl, combine the flour, sugar, and salt. Make a well in the center and pour in the yeast mixture and olive oil.
3. Mix the ingredients together until a dough 

In [ ]:
# # Serialization (DOES NOT WORK ANYMORE)
# from langchain.chat_models import ChatOpenAI
# from langchain.llms.openai import OpenAI
# from langchain.llms.loading import load_llm


# chat = ChatOpenAI(
#     temperature=0.1,
#     max_tokens=450,
#     model="gpt-3.5-turbo-16k"
# )

# chat.save("model.json") #model저장
# #chat = load_llm("model.json")

### Memory : 

In [2]:
# Conversation Buffer Memory : save whole conversation.
# use when text completion, but for chat model, you need 
from langchain.memory import ConversationBufferMemory

# 1. Create memory (chat purpose or not)
memory = ConversationBufferMemory(return_messages=True) #return_messages를 넣어줘야지 Chat용도
# 2. Save
memory.save_context({"input":"Hi!"},{"output":"How are you?"})
# 3. load
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

In [3]:
memory.save_context({"input":"Hi!"},{"output":"How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}

In [4]:
memory.save_context({"input":"Hi!"},{"output":"How are you?"})

memory.load_memory_variables({})
# it keeps store the same thing over and over agian. 

{'history': [HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?'),
  HumanMessage(content='Hi!'),
  AIMessage(content='How are you?')]}

In [6]:
#ConversationBufferWindowMemory (up to a point ex) last 5 message) 
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input,output):
    memory.save_context({"input":input},{"output":output})
    
add_message(1,1)

In [7]:
add_message(2,2)
add_message(3,3)
add_message(4,4)

In [8]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

In [9]:
add_message(5,5)
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [10]:
#ConversationSummaryMemory : 
# It doesn't save messages as they come, instead it makes summary of the conversation. 

from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input,output):
    memory.save_context({"input":input},{"outputs":output})

def get_history():
    return memory.load_memory_variables({})

In [11]:
add_message("Hi, I am Chulwon, I live in Chicago","Wow! That is so cool!")
add_message("Chicago is so pretty","I wish I could go!!")

In [12]:
get_history()

{'history': 'Chulwon from Chicago introduces themselves to the AI, who responds enthusiastically. The human mentions how pretty Chicago is, and the AI expresses a desire to visit the city.'}

In [18]:
#ConversationSummaryBufferMemory :
# It keeps count of how many messages we have sending to the memory.
# Once it hit the limit, it will start summarizing the old messages.

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limt=15, # how many maximum tokens can our messages be before they become summarized. 
    return_messages=True
) 

def add_message(input,output):
    memory.save_context({"input":input},{"outputs":output})

def get_history():
    return memory.load_memory_variables({})

In [19]:
add_message("Hi, I am Chulwon, I live in Chicago","Wow! That is so cool!")

In [20]:
get_history()

{'history': [HumanMessage(content='Hi, I am Chulwon, I live in Chicago'),
  AIMessage(content='Wow! That is so cool!')]}

In [21]:
add_message("Chicago is so pretty","I wish I could go!!")

In [22]:
get_history()

{'history': [HumanMessage(content='Hi, I am Chulwon, I live in Chicago'),
  AIMessage(content='Wow! That is so cool!'),
  HumanMessage(content='Chicago is so pretty'),
  AIMessage(content='I wish I could go!!')]}

In [23]:
add_message("How far is Chicago from Atlanta?","I don't know! Not that far!")

In [24]:
get_history()

{'history': [HumanMessage(content='Hi, I am Chulwon, I live in Chicago'),
  AIMessage(content='Wow! That is so cool!'),
  HumanMessage(content='Chicago is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Chicago from Atlanta?'),
  AIMessage(content="I don't know! Not that far!")]}

In [25]:
add_message("How far is New Yrok from Korea?","I don't know! Not that far!")

In [26]:
get_history()

{'history': [HumanMessage(content='Hi, I am Chulwon, I live in Chicago'),
  AIMessage(content='Wow! That is so cool!'),
  HumanMessage(content='Chicago is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Chicago from Atlanta?'),
  AIMessage(content="I don't know! Not that far!"),
  HumanMessage(content='How far is New Yrok from Korea?'),
  AIMessage(content="I don't know! Not that far!")]}

In [27]:
add_message("How far is Tokyo from Korea?","I don't know! Not that far!")

In [28]:
get_history()

{'history': [HumanMessage(content='Hi, I am Chulwon, I live in Chicago'),
  AIMessage(content='Wow! That is so cool!'),
  HumanMessage(content='Chicago is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Chicago from Atlanta?'),
  AIMessage(content="I don't know! Not that far!"),
  HumanMessage(content='How far is New Yrok from Korea?'),
  AIMessage(content="I don't know! Not that far!"),
  HumanMessage(content='How far is Tokyo from Korea?'),
  AIMessage(content="I don't know! Not that far!")]}

In [29]:
add_message("How far is Rome from Korea?","I don't know! Not that far!")
get_history()

{'history': [HumanMessage(content='Hi, I am Chulwon, I live in Chicago'),
  AIMessage(content='Wow! That is so cool!'),
  HumanMessage(content='Chicago is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Chicago from Atlanta?'),
  AIMessage(content="I don't know! Not that far!"),
  HumanMessage(content='How far is New Yrok from Korea?'),
  AIMessage(content="I don't know! Not that far!"),
  HumanMessage(content='How far is Tokyo from Korea?'),
  AIMessage(content="I don't know! Not that far!"),
  HumanMessage(content='How far is Rome from Korea?'),
  AIMessage(content="I don't know! Not that far!")]}

In [30]:
add_message("what is the tallest building in Korea?","I don't know!!")
get_history()

{'history': [HumanMessage(content='Hi, I am Chulwon, I live in Chicago'),
  AIMessage(content='Wow! That is so cool!'),
  HumanMessage(content='Chicago is so pretty'),
  AIMessage(content='I wish I could go!!'),
  HumanMessage(content='How far is Chicago from Atlanta?'),
  AIMessage(content="I don't know! Not that far!"),
  HumanMessage(content='How far is New Yrok from Korea?'),
  AIMessage(content="I don't know! Not that far!"),
  HumanMessage(content='How far is Tokyo from Korea?'),
  AIMessage(content="I don't know! Not that far!"),
  HumanMessage(content='How far is Rome from Korea?'),
  AIMessage(content="I don't know! Not that far!"),
  HumanMessage(content='what is the tallest building in Korea?'),
  AIMessage(content="I don't know!!")]}

In [31]:
#ConversationKGMemory : it builds knowledge graph of the entity

from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Chulwon, I live in Chicago", "Wow that is so cool!")

In [32]:
memory.load_memory_variables({"input": "who is Chulwon?"})

{'history': [SystemMessage(content='On Chulwon: Chulwon lives in Chicago.')]}

In [33]:
add_message("Chulwon likes steak", "Wow that is so cool!")

In [34]:
memory.load_memory_variables({"inputs": "what does Chulwon like"})

{'history': [SystemMessage(content='On Chulwon: Chulwon lives in Chicago. Chulwon likes steak.')]}

### Memory on LLMChain

In [39]:
# LLMChain : off the shelf chain(general purpose chain) but hard to customize.
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
)


chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True
)

chain.predict(question='My name is Chulwon')



> Entering new LLMChain chain...
Prompt after formatting:
My name is Chulwon

> Finished chain.


'Nice to meet you, Chulwon! How can I assist you today?'

In [40]:
chain.predict(question='I live in Seoul')



> Entering new LLMChain chain...
Prompt after formatting:
I live in Seoul

> Finished chain.


", the capital city of South Korea. It is a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic temples. I love exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain. Seoul is a dynamic city that never fails to impress me with its energy and charm."

In [41]:
chain.predict(question='What is my name?')

#verbose true로 보니까 history 가 없어



> Entering new LLMChain chain...
Prompt after formatting:
What is my name?

> Finished chain.


"I'm sorry, I do not have access to personal information such as your name."

In [42]:
# 메모리는 업데이트가 된거는 맞아..the only problem is llm is not being given the history of the conversation.
memory.load_memory_variables({})

{'history': "System: The human introduces themselves as Chulwon and mentions they live in Seoul, the capital city of South Korea. The AI responds by describing Seoul as a bustling metropolis with a vibrant culture, delicious food, and a mix of modern skyscrapers and historic temples. The AI expresses their love for exploring the city's neighborhoods, trying new restaurants, and taking in the beautiful views from Namsan Mountain, highlighting Seoul's dynamic energy and charm.\nHuman: What is my name?\nAI: I'm sorry, I do not have access to personal information such as your name."}

In [48]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",#; 메모리는 잘 저장되는 것을 위에 확인 했으니, 아래 템플릿에 위 memory를 잘 넣어줘야함 
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

chain.predict(question='My name is Chulwon')



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human:My name is Chulwon
    You:


> Finished chain.


'Hello Chulwon! How can I assist you today?'

In [49]:
chain.predict(question='I live in Seoul')



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Chulwon
AI: Hello Chulwon! How can I assist you today?
    Human:I live in Seoul
    You:


> Finished chain.


"That's great to know! How can I assist you with anything related to Seoul or any other topic today?"

In [50]:
chain.predict(question='What is my name?')



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: My name is Chulwon
AI: Hello Chulwon! How can I assist you today?
Human: I live in Seoul
AI: That's great to know! How can I assist you with anything related to Seoul or any other topic today?
    Human:What is my name?
    You:


> Finished chain.


'Your name is Chulwon.'

### ChatBased Memory

In [51]:
# Mmeoery classes can be output in 2 format. 1. string 2. messages 
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True #dont turn it into string
)

prompt= ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"), #누가 보냈는지 알 수 없는, 예측하기 어려운 메세지의 양과 제한없는 양의 메새지를 가질수 있음. 
    ("human","{question}")
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(question='My name is Chulwon')



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Chulwon

> Finished chain.


'Nice to meet you, Chulwon! How can I assist you today?'

### LCEL Based Memory

In [52]:
# more control to memory setting
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm #
# 1. load memory()돌아가면서, 그 output이 history 에 들어가고  
# 2. 그게 사용자의 input과 결합되면서 prompt에 전달

def invoke_chain(question): # 이게 필요한 이유는, 우리가 메모리 관리를 수동으로 하고 있기 떄문에 우리는 사용자와 기계간의 인터렉션을 메모리에 저쟁해야함
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [53]:
invoke_chain("My name is Chulwon")

content='Nice to meet you, Chulwon! How can I assist you today?'


In [54]:
invoke_chain("What is my name?")

content='Your name is Chulwon.'


### Recap of Memory
* Purpose is adding the history to the prompt that LLM gets.
* LLM has off the shelf(automatic) 
* Langchain expression : manual but more customize

3 ways for Memeory
- 1. LLMChain
- 2. Chatprompt template
- 3. Manual (LCEL)